<a href="https://colab.research.google.com/github/donghwi2022/ds-section1-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 다음 분기에 어떤 게임을 설계해야 할까

# **Part 1. 필요 라이브러리 및 데이터 로드**
- 필요 라이브러리 import 및 한글 깨짐문제 해결
- csv 파일 load

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
#한글 깨짐문제 해결
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumBarunGothic');

In [ ]:
# url로 필요 csv 파일 불러오기
url = "https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv"
df = pd.read_csv(url)
df.head()

# **Part 2. 데이터 전처리 진행**

In [ ]:
df.info()

In [ ]:
# 첫번째 column은 index를 나타내므로 해당 열 제거
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head()

In [ ]:
# 결측치 확인
df.isnull().sum()

In [ ]:
# 데이터 양이 충분하므로, 결측치가 존재하는 데이터 삭제
df.dropna(inplace = True)
df.info()

In [ ]:
# 중복값 확인(게임 명, 지원되는 플랫폼, 출시 연도, 장르, 게임회사 명이 같은 row는 중복데이터로 간주)
df[df.duplicated(['Name', 'Platform', 'Year', 'Genre', 'Publisher'])]

In [ ]:
# 중복되는 데이터 확인
df[(df.Name == 'Madden NFL 13') & (df.Platform == 'PS3') & (df.Year == 2012.0) & (df.Genre == 'Sports') & (df.Publisher == 'Electronic Arts')]

In [ ]:
# 중복되는 데이터 중 정확한 데이터를 알 수 없기 때문에 해당 row를 전부 삭제
duplicate_index = df[df.duplicated(['Name', 'Platform', 'Year', 'Genre', 'Publisher'])].index
duplicated_all_index = []
for i in range(len(duplicate_index)) :
    duplicated_data = df.loc[duplicate_index[i]].Name, df.loc[duplicate_index[i]].Platform, df.loc[duplicate_index[i]].Year, df.loc[duplicate_index[i]].Genre, df.loc[duplicate_index[i]].Publisher
    duplicated_all_index.extend(df[(df.Name == duplicated_data[0]) & (df.Platform == duplicated_data[1]) & (df.Year == duplicated_data[2]) & (df.Genre == duplicated_data[3]) & (df.Publisher == duplicated_data[4])].index)
    # append를 사용할 경우 배열에 배열을 추가할 시, 배열이 그대로 유지되지만 extend는 쪼개서 한 개씩 들어감
df.drop(duplicated_all_index, inplace = True)
df.info()

In [ ]:
# 출고량에서 숫자가 아닌 데이터 출력
for i in range(5, 9) : 
    df_1 = df.iloc[:, i].str.extract(r'([^\.\d])')
    print(df.columns[i], df_1[0].unique())


In [ ]:
# 출고량 단위 맞춤
cols = df.columns[5:9] # 출고량에 해당하는 column 명 저장

# K 단위를 M 단위로 맞추기 위함 함수 작성
def k_to_m(x) : 
    return x/1000 

for i in cols : 
    df[i] = df[i].str.replace('M', '') # M은 문자만 삭제
    df1 = df[df.loc[:, i].str.contains('K')].loc[:, i].str.replace('K', '').astype(int).apply(k_to_m) # K가 들어가는 데이터를 단위에 맞춰서 변형한 후 df1에 저장
    for j in df1.index : 
        df.loc[j, i] = df1.loc[j] # df1의 값을 바탕으로 df의 값을 변경
    df[i] = df[i].astype(float) # df의 타입 변경
df.head(11)

In [ ]:
# 연도도 정수형태로 변경 후, 연도의 데이터를 확인
df['Year'] = df['Year'].astype(int)
df.Year.unique()

In [ ]:
# 연도의 형식에 맞지 않는 데이터 변경
def year(x) :
    if x < 22 :
        return x + 2000
    elif (x >= 22) & (x < 100) :
        return x + 1900
    else :
        return x
df['Year'] = df['Year'].apply(year)
df.Year.unique()

In [ ]:
# 게임의 출고량 총합을 나타내는 column 작성
df['Sales_Total'] = df['EU_Sales'] + df['JP_Sales'] + df['NA_Sales'] + df['Other_Sales']

In [ ]:
# 데이터프레임 인덱스 재설정
df.reset_index(drop=True, inplace = True)

In [ ]:
# 최종 데이터 형태 확인
df.info()

In [ ]:
df.head()

# **Part 3. 다음 분기에 설계할 게임에 대한 기획 진행**
## 게임 회사는 'Idea Factory'
## 지금이 2016년도라 생각하고 2017년도에 설계할 게임에 대한 기획 진행
   (2017년부터는 데이터가 거의 없기 때문에)

In [ ]:
# df = 전체 데이터, df_co = 내가 정한 회사의 데이터
# 다음 분기를 계획하는 것이기 때문에 최근 10년의 데이터를 바탕으로 기획 진행
df_all = df[(df.Year >= 2012) & (df.Year < 2017)].reset_index(drop=True)
df_co = df[(df.Year >= 2012) & (df.Year < 2017) & (df.Publisher == 'Idea Factory')].reset_index(drop=True)
df_co.info()

In [ ]:
df_co.head()

In [ ]:
df_all.info()

In [ ]:
df_all.head()

## **Question 1. 지역에 따라 선호하는 게임 장르가 다를까**

In [ ]:
# 해당 함수 작성
def genre_by_region(df) :
    df_by_Genre = df.groupby(['Genre'])[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].sum() # 장르로 그룹화
    df_by_region = df_by_Genre.transpose() # 지역별로 확인하기 전치행렬

    #시각화
    df_by_region.plot.bar(figsize=(10,6))
    plt.xlabel('지역', size = 20)
    plt.xticks(rotation = 0, size = 15)
    plt.ylabel('출고량\n(단위 : 백만)', size = 15, rotation = 0, labelpad = 40)
    plt.suptitle('지역별 장르의 출고량', size = 25)
    if len(df) <= 100 : 
        plt.title('by Idea Factory', size = 15)
    else : 
        plt.title('by 모든 게임회사', size = 15)
    plt.show()

In [ ]:
genre_by_region(df_co)

In [ ]:
genre_by_region(df_all)

### + 위의 그래프를 통해 해답을 얻을 수 있지만, 가설 검증을 통해 한 번 더 확인해보기


지역에 따라 선호하는 장르에 차이가 있는지를 출고량을 통해 확인하려고 한다. <br>
같은 장르일때 지역에 따라 출고량이 같으면 선호하는 장르가 같다고 생각한다.

In [ ]:
# 지역에 따라 같은 장르의 출고량에 차이가 있는지 확인하는 함수 작성
def compare_sales (sample1, sample2) : 
    # 등분산성 여부 확인
    if stats.levene(sample1, sample2).pvalue < 0.05 :
        equal_var_value = False
    else :
        equal_var_value = True
    statistic, p_value = stats.ttest_ind(sample1, sample2, equal_var = equal_var_value)
    pvalue = round(p_value, 2)
    if pvalue >= 0.05 :
        print('pvalue : {}'.format(pvalue))
        print('pvalue가 0.05 이상이므로 귀무가설을 기각하지 않고 귀무가설을 택한다.')
    else :
        print('pvalue : {}'.format(pvalue))
        print('pvalue가 0.05 미만이므로 귀무가설을 기각하고 대립가설을 택한다.')

귀무가설 : Action 장르의 북미 출고량과 유럽 출고량에 차이가 없다.<br>
대립가설 : Action 장르의 북미 출고량과 유럽 출고량에 차이가 있다.

In [ ]:

na_sales = df_all[df_all['Genre'] == 'Action'].NA_Sales
eu_sales = df_all[df_all['Genre'] == 'Action'].EU_Sales

compare_sales (na_sales, eu_sales)

귀무가설 : Action 장르의 유럽 출고량과 일본 출고량에 차이가 없다.<br>
대립가설 : Action 장르의 유럽 출고량과 일본 출고량에 차이가 있다.

In [ ]:
eu_sales = df_all[df_all['Genre'] == 'Action'].EU_Sales
jp_sales = df_all[df_all['Genre'] == 'Action'].JP_Sales

compare_sales (eu_sales, jp_sales)

결론 : Action 장르의 유럽 출고량과 일본 출고량에 차이가 있기 때문에 지역에 따라 선호하는 장르가 다르다고 할 수 있다.

## **Question 2. 연도별 게임의 트렌드가 있을까**

### 연도별 장르에 변화가 있는지 확인

In [ ]:
df1 = df_all.groupby(['Genre', 'Year'], as_index=False)[['Sales_Total']].sum() # 장르와 연도로 그룹화
df1.head()

In [ ]:
genre_list = df1.Genre.unique()
plt.figure(figsize = (10, 6))
for i in range(len(genre_list)) :
    plt.plot(df1[df1.Genre == genre_list[i]]['Year'], df1[df1.Genre == genre_list[i]]['Sales_Total'], label = genre_list[i])
plt.xlabel('연도', size = 20)
plt.ylabel('출고량\n(단위 : 백만)', size = 15, rotation = 0, labelpad = 40)
plt.xticks(range(2012, 2017, 1))
plt.legend()
plt.title('연도에 따른 장르별 출고량 변화', size = 20)
plt.show()

### 연도별 플랫폼에 변화가 있는지 확인

In [ ]:
df2 = df_all.groupby(['Platform', 'Year'], as_index=False)[['Sales_Total']].sum() # 플랫폼과 연도로 그룹화
df2.head()

In [ ]:
platform_list = df2.Platform.unique()
plt.figure(figsize = (10, 6))
for i in range(len(platform_list)) :
    plt.plot(df2[df2.Platform == platform_list[i]]['Year'], df2[df2.Platform == platform_list[i]]['Sales_Total'], label = platform_list[i])
plt.xlabel('연도', size = 20)
plt.ylabel('출고량\n(단위 : 백만)', size = 15, rotation = 0, labelpad = 40)
plt.xticks(range(2012, 2017, 1))
plt.legend()
plt.title('연도에 따른 플랫폼별 출고량 변화', size = 20)
plt.show()

## **Question 3. 출고량이 높은 게임에 대한 분석 및 시각화**

### 우리 회사의 데이터에서 분석 및 시각화

In [ ]:
# Idea Factory의 게임을 출고량이 높은 순서대로 게임 이름을 정렬
df_sales_co = df_co.groupby(['Name', 'Genre'], as_index=False)[['Sales_Total']].sum()
df_sales_co.sort_values('Sales_Total', ascending = False, inplace = True)
df_sales_co

In [ ]:
df_co_sales_top = df_sales_co[(df_sales_co.Name == 'Hyperdevotion Noire: Goddess Black Heart') | (df_sales_co.Name == 'Code:Realize - Sousei no Himegimi') 
                              | (df_sales_co.Name == 'Makai Shin Trillion') | (df_sales_co.Name == 'Amnesia Later') 
                              | (df_sales_co.Name == 'Hakuouki: Yuugi Roku Ni - Matsuri Hayashi to Taishitachi')]
df_co_sales_top

In [ ]:
sns.barplot(x="Genre", y="Sales_Total", hue="Name", data=df_co_sales_top)
plt.title("장르별 게임의 출고량")
plt.legend(bbox_to_anchor = (1,1))
plt.show()

### 전체 게임의 데이터에서 분석 및 시각화

In [ ]:
# 전체 게임을 출고량이 높은 순서대로 게임 이름을 정렬
df_sales = df_all.groupby(['Name'], as_index=False)[['Sales_Total']].sum()
df_sales.sort_values('Sales_Total', ascending = False, inplace = True)
df_sales

In [ ]:
df_all_sales_top = df_all[(df_all.Name == 'Grand Theft Auto V') | (df_all.Name == 'Call of Duty: Black Ops II') | (df_all.Name == 'Call of Duty: Ghosts')]
df_all_sales_top

In [ ]:
# 플랫폼별 게임의 출고량 시각화
df_by_Platform = df_all_sales_top.groupby(['Platform', 'Name'], as_index=False)[['Sales_Total']].sum() # 장르로 그룹화

sns.barplot(x="Name", y="Sales_Total", hue="Platform", data=df_by_Platform)
plt.title("게임의 플랫폼별 출고량")
plt.legend(bbox_to_anchor = (1,1))
plt.show()

In [ ]:
# 장르별 게임의 출고량 시각화
df_by_Genre = df_all_sales_top.groupby(['Genre', 'Name'], as_index=False)[['Sales_Total']].sum() # 장르로 그룹화

sns.barplot(x="Genre", y="Sales_Total", hue="Name", data=df_by_Genre)
plt.title("장르별 게임의 출고량")
plt.legend(bbox_to_anchor = (1,1))
plt.show()

## **결과 요약**
2016년 기준 <br>
1. 플랫폼별 출고량 순위
    1. PS4
    2. XOne
    3. 3DS

2. 장르별 출고량 순위
    1. Action
    2. Shooter
    3. Sports <br>

우리 회사(Idea Factory)의 출고량 순위 (괄호 안은 장르를 나타냄)
1. Hyperdevotion Noire: Goddess Black Heart (Role-Playing)
2. Code:Realize - Sousei no Himegimi (Adventure)
3. Makai Shin Trillion (Role-Playing)
4. Amnesia Later (Misc)
5. Hakuouki: Yuugi Roku Ni - Matsuri Hayashi to Taishitachi (Action)

## **결론**
1. 2015년 이후로 게임 장르 및 지원되는 플랫폼에 따른 출고량이 하락하는 경향이 보입니다.<br>
2. 하지만, Action 장르의 게임이 출고량이 가장 많고 심지어 출고량 1위의 게임 장르가 2,3위의 출고량과 차이가 많이 나는 것을 확인할 수 있습니다.
3. 그리고 우리 회사도 회사 게임 내 5위에 머무르긴 했지만 Action 장르의 게임을 개발한 것을 확인할 수 있습니다.
4. 플랫폼은 PS4, XOne,3DS 가 출고량이 가장 많은 것을 확인할 수 있습니다.

따라서, 다음 분기에는 Action 장르의 게임을 개발하는데 'PS4, XOne, 3DS' 의 플랫폼을 지원할 수 있도록 고려해야 합니다.